![Image](actinia_logo.png)

## Introduction

Actinia is a REST service to process geographical data that can be
managed by the GRASS GIS software system. The software is designed to
expose a GRASS GIS database and many [GRASS GIS](https://grass.osgeo.org/) processing tools as a
[REST service](https://en.wikipedia.org/wiki/Representational_State_Transfer).
Hence, access to GRASS resources like raster maps,
space-time raster datasets, processing and analysis modules are
available via URL. In addition, actinia allows the cloud based processing
of data, for example all Landsat 4-8 scenes as well as all
Sentinel-2 scenes in an ephemeral database. The computational results
of ephemeral processing are available via object storage as GeoTIFF/COG
or GeoPackage files.

The actinia service consists of the *[actinia core](https://github.com/actinia-org/actinia-core)*
that provides the basic but sophisticated processing service and
*[actinia plugins](https://github.com/orgs/mundialis/repositories?q=actinia+plugins&type=all&language=&sort=)*
that provide problem specific services like NDVI computation from Sentinel-2
or Landsat data, spatio-temporal statistical analysis and many more.

The following example is a Jupyter Notebook version of the online
[actinia tutorial](https://actinia-dev.mundialis.de/tutorial/introduction.html). 

### What is REST?

The Representational state transfer ([REST](https://en.wikipedia.org/wiki/Representational_State_Transfer))
is an architectural style based on [HTTP](https://en.wikipedia.org/wiki/Hypertext_Transfer_Protocol)
that uses the [request methods](https://en.wikipedia.org/wiki/Hypertext_Transfer_Protocol#Request_methods)
GET, DELETE, POST, and PUT to manipulate and receive resources with stateless operations.

While GET requests can be send easily from a browser, POST, PUT or
DELETE request can not. To access the full potential of actinia you will
need a HTTP client, that talks all HTTP communication methods. Here we will
use this Jupyter Notebook for this purpose.

---

### actinia API documentation

* [Stable actinia API v3 docs](https://redocly.github.io/redoc/?url=https://actinia.mundialis.de/api/v3/swagger.json)
* [Development actinia API v3 docs](https://redocly.github.io/redoc/?url=https://actinia-dev.mundialis.de/api/v3/swagger.json)

---

### Requirements

#### Software & Modules

This tutorial assumes your are comfortable with the [Python](https://python.org) programming language. Familiarity with basic REST API concepts and usage is also assumed.

Python modules used in this tutorial are:
* [requests](http://docs.python-requests.org/)
* [json](https://docs.python.org/3/library/json.html)


#### Actinia API user and password

This demo requires credentials for authentication set below in **Preparation** as a variable. Another actinia instance might require different credentials.

### Helper Modules and Functions
Before interacting with the actinia server using Python, we will import required packages an set up a helper function to print formatted JSON using json.

***Note:*** *You may need to install two helpful browser plugins called **RESTman** and **JSON Formatter** that format JSON and makes it easier to read:*

* [RESTman extension](https://chrome.google.com/webstore/detail/restman/ihgpcfpkpmdcghlnaofdmjkoemnlijdi)
* [JSON Formatter](https://chrome.google.com/webstore/detail/json-formatter/bcjindcccaagfpapjjmafapmmgkkhgoa)

## Preparation


In [ ]:
# first, let's import the required packages.

from pprint import pprint
import sys
import json
import time

import requests
from requests.auth import HTTPBasicAuth


To simplify our life in terms of server communication we store the credentials and REST server URL in  variables.

In [ ]:
# variables to set the actinia host, version, and user

actinia_baseurl = "https://actinia.mundialis.de"
actinia_version = "v3"
actinia_url = actinia_baseurl + "/api/" + actinia_version
actinia_auth = HTTPBasicAuth('demouser', 'gu3st!pa55w0rd')

In [ ]:
# helper function to print formatted JSON using the json module

def print_as_json(data):
    print(json.dumps(data, indent=2))

# helper function to verify a request
def verify_request(request, success_code=200):
    if request.status_code != success_code:
        print("ERROR: actinia processing failed with status code %d!" % request.status_code)
        print("See errors below:")
        print_as_json(request.json())
        request_url = request.json()["urls"]["status"]
        requests.delete(url=request_url, auth=actinia_auth)
        raise Exception("The resource <%s> has been terminated." % request_url)

## Examples

* Data management
* Landsat and Sentinel-2 NDVI computation

### Data management

List all locations that are available in the actinia persistent database:

In [ ]:
# make a GET request to the actinia data API
request_url = actinia_url + "/locations"
print("actinia GET request:")
print(request_url)
print("---")
request = requests.get(url=request_url, auth=actinia_auth)

# check if anything went wrong
verify_request(request, 200)

# get a json-encoded content of the response
jsonResponse = request.json()

print("Available locations:")

# print formatted JSON
print_as_json(jsonResponse)

#### List mapsets in locations

Next, we look at so-called **"mapsets"** which are subfolders in a location (just to better organise the geospatial data).

List all **mapsets** inside the location `nc_spm_08`:

In [ ]:
# make a GET request to the actinia data API
request_url = actinia_url + "/locations/nc_spm_08/mapsets"
print("actinia GET request:")
print(request_url)
print("---")
request = requests.get(url=request_url, auth=actinia_auth)

# check if anything went wrong
verify_request(request, 200)

# get a json-encoded content of the response
jsonResponse = request.json()

print("Mapsets in nc_spm_08 location:")

# print formatted JSON
print_as_json(jsonResponse["process_results"])

Now list all **mapsets** inside the location `latlong_wgs84`:

In [ ]:
# make a GET request to the actinia data API
request_url = actinia_url + "/locations/latlong_wgs84/mapsets"
print("actinia GET request:")
print(request_url)
print("---")
request = requests.get(url=request_url, auth=actinia_auth)

# check if anything went wrong
verify_request(request, 200)

# get a json-encoded content of the response
jsonResponse = request.json()

print("Mapsets in latlong_wgs84 location:")

# print formatted JSON
print_as_json(jsonResponse["process_results"])

#### List contents of a mapset

List all **raster layers** in location `latlong_wgs84` and mapset `Sentinel2A`:

In [ ]:
# make a GET request to the actinia data API
request_url = actinia_url + "/locations/latlong_wgs84/mapsets/Sentinel2A/raster_layers"
print("actinia GET request:")
print(request_url)
print("---")
request = requests.get(url=request_url, auth=actinia_auth)

# check if anything went wrong
verify_request(request, 200)

# get a json-encoded content of the response
jsonResponse = request.json()

print("Raster layers in mapset Sentinel2A of location latlong_wgs84:")

# print formatted JSON
print_as_json(jsonResponse["process_results"])

List all **space-time raster datasets (STRDS)** in location `ECAD` and mapset `PERMANENT`:

In [ ]:
# make a GET request to the actinia data API
request_url = actinia_url + "/locations/ECAD/mapsets/PERMANENT/strds"
print("actinia GET request:")
print(request_url)
print("---")
request = requests.get(url=request_url, auth=actinia_auth)

# check if anything went wrong
verify_request(request, 200)

# get a json-encoded content of the response
jsonResponse = request.json()

print("Space-time raster datasets (STRDS) in location ECAD and mapset PERMANENT:")

# print formatted JSON
print_as_json(jsonResponse["process_results"])

List all **raster map layers of the STRDS** precipitation_1950_2013_yearly_mm:

In [ ]:
# make a GET request to the actinia data API
request_url = actinia_url + "/locations/ECAD/mapsets/PERMANENT/strds/precipitation_1950_2013_yearly_mm/raster_layers"
print("actinia GET request:")
print(request_url)
print("---")
request = requests.get(url=request_url, auth=actinia_auth)

# check if anything went wrong
verify_request(request, 200)

# get a json-encoded content of the response
jsonResponse = request.json()

print("Raster map layers of the STRDS precipitation_1950_2013_yearly_mm:")

# print formatted JSON
raster_layers = [i["id"] for i in jsonResponse["process_results"]]
print_as_json(raster_layers)

---

### Landsat and Sentinel-2 NDVI computation

This API call will compute the NDVI of the top of atmospheric corrected (TOAR) Landsat-4 scene LC80440342016259LGN00:

In [ ]:
# make a POST request to the actinia data API
request_url = actinia_url + "/landsat_process/LC80440342016259LGN00/TOAR/NDVI"
print("actinia POST request:")
print(request_url)
print("---")
request = requests.post(url=request_url, auth=actinia_auth)

# check if anything went wrong
verify_request(request, 200)

# get a json-encoded content of the response
jsonResponse = request.json()

print("Waiting for the process to finish...\n")

if jsonResponse["status"] in ("accepted", "running"):
    # poll the resource until status has changed
    request_url = jsonResponse["urls"]["status"]
    request = requests.get(url=request_url, auth=actinia_auth)
    jsonResponse = request.json()

    while jsonResponse["status"] in ("accepted", "running"):
        time.sleep(1)
        jsonResponse = request.json()

    # check if anything went wrong
    verify_request(request, 200)

    print("NDVI of the top of atmospheric corrected (TOAR) Landsat-4 scene LC80440342016259LGN00:")
    # print formatted JSON
    print_as_json(jsonResponse)

NDVI computation of Sentinel-2 scene S2A_MSIL1C_20170212T104141_N0204_R008_T31TGJ_20170212T104138:

In [ ]:
# make a POST request to the actinia data API
request_url = actinia_url + "/sentinel2_process/ndvi/S2A_MSIL1C_20170212T104141_N0204_R008_T31TGJ_20170212T104138"
print("actinia POST request:")
print(request_url)
print("---")
request = requests.post(url=request_url, auth=actinia_auth)

# check if anything went wrong
verify_request(request, 200)

# get a json-encoded content of the response
jsonResponse = request.json()

print("Waiting for the process to finish...\n")

if jsonResponse["status"] in ("accepted", "running"):
    # poll the resource until status has changed
    request_url = jsonResponse["urls"]["status"]
    request = requests.get(url=request_url, auth=actinia_auth)
    jsonResponse = request.json()

    while jsonResponse["status"] in ("accepted", "running"):
        time.sleep(1)
        jsonResponse = request.json()

    # check if anything went wrong
    verify_request(request, 200)

    print("NDVI computation of Sentinel-2 scene S2A_MSIL1C_20170212T104141_N0204_R008_T31TGJ_20170212T104138:")
    # print formatted JSON
    print_as_json(jsonResponse)